In this homework, we're going to combine data from various sources to process it in Pandas and generate additional fields.

If not stated otherwise, please use the [LINK][link] covered at the livestream to re-use the code snippets.

[link]: https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/02-dataframe-analysis/%5B2025%5D_Module_02_Colab_Working_with_the_data.ipynb
---

# Question 1: [IPO] Withdrawn IPOs by Company Type

What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?

From the withdrawn IPO list ([stockanalysis.com/ipos/withdrawn][wdipo]), collect and process the data to find out which company type saw the most withdrawn IPO value.
Steps:

1. Use `pandas.read_html()` with the URL above to load the IPO withdrawal table into a DataFrame.
   _It is a similar process to Code Snippet 1 discussed at the livestream._ You should get **99 entries**.
2. Create a new column called `Company Class`, categorizing company names based on patterns like:
    - "Acquisition Corp" or "Acquisition Corporation" → `Acq.Corp`
    - "Inc" or "Incorporated" → `Inc`
    - "Group" → `Group`
    - "Ltd" or "Limited" → `Limited`
    - "Holdings" → `Holdings`
    - Others → `Other`
   
   Order: Please follow the listed order of classes and assign the first matched value (e.g., for 'shenni holdings limited',
   you assign the `Limited` class).
   
    Hint: make your function more robust by converting names to lowercase and splitting into words before matching patterns.
4. Define a new field `Avg. price` by parsing the `Price Range` field (create a function and apply it to the `Price` Range column).
   Examples: 
   - '$8.00-$10.00' → `9.0`
   - '$5.00' → `5.0`
   - '-' → `None`
5. Convert Shares Offered to numeric, clean missing or invalid values.
6. Create a new column:
   `Withdrawn Value = Shares Offered * Avg Price` **(71 non-null values)**
7. Group by Company Class and calculate total withdrawn value.
8. **Answer**: Which class had the highest total value of withdrawals?

[wdipo]: https://stockanalysis.com/ipos/withdrawn/

In [1]:
from bs4 import BeautifulSoup
from polars import col as c
import polars as pl
import requests as r

def read_table(url: str) -> pl.DataFrame:
    resp = r.get(url)
    doc = BeautifulSoup(resp.content, 'html.parser')
    table = doc.find(id='main-table')
    keys = [e.text.strip() for e in table.find_all(name='th')]

    return pl.DataFrame([
        dict(zip(keys, [e.text for e in row.find_all(name='td')]))
        for row in table.find(name='tbody').find_all(name='tr')
    ])

In [2]:
df = read_table('https://stockanalysis.com/ipos/withdrawn')
df.shape

(100, 4)

In [3]:
df.head()

Symbol,Company Name,Price Range,Shares Offered
str,str,str,str
"""ODTX""","""Odyssey Therapeutics, Inc.""","""-""","""-"""
"""UNFL""","""Unifoil Holdings, Inc.""","""$3.00 - $4.00""","""2,000,000"""
"""AURN""","""Aurion Biotech, Inc.""","""-""","""-"""
"""ROTR""","""PHI Group, Inc.""","""-""","""-"""
"""ONE""","""One Power Company""","""-""","""-"""


In [4]:
df = (
    df.with_columns(
        c('Shares Offered').str.replace_all(',', '').str.to_integer(strict=False),
        (
            c('Price Range').str.replace_all(r'\$', '')
            .str.split(' - ')
            .list.eval(pl.element().cast(pl.Float32, strict=False))
            .list.mean()
        ).alias('Avg. price'),
        pl.when(c('Company Name').str.contains_any(['acquisition corp'], ascii_case_insensitive=True)).then(pl.lit('Acq.Corp'))
        .when(c('Company Name').str.contains_any(['inc'],                ascii_case_insensitive=True)).then(pl.lit('Inc'))
        .when(c('Company Name').str.contains_any(['group'],              ascii_case_insensitive=True)).then(pl.lit('Group'))
        .when(c('Company Name').str.contains_any(['ltd', 'limited'],     ascii_case_insensitive=True)).then(pl.lit('Limited'))
        .when(c('Company Name').str.contains_any(['holdings'],           ascii_case_insensitive=True)).then(pl.lit('Holdings'))
        .otherwise(pl.lit('Other'))
        .cast(pl.Categorical(ordering='lexical'))
        .alias('Company Class')
    )
    .with_columns(
        (c('Avg. price') * c('Shares Offered') / 1_000_000.0).alias('Withdrawn Value (million)')
    )
    .filter(c('Withdrawn Value (million)').is_not_null())
)
df

Symbol,Company Name,Price Range,Shares Offered,Avg. price,Company Class,Withdrawn Value (million)
str,str,str,i64,f32,cat,f64
"""UNFL""","""Unifoil Holdings, Inc.""","""$3.00 - $4.00""",2000000,3.5,"""Inc""",7.0
"""HPOT""","""The Great Restaurant Developme…","""$4.00 - $6.00""",1400000,5.0,"""Limited""",7.0
"""CABR""","""Caring Brands, Inc.""","""$4.00""",750000,4.0,"""Inc""",3.0
"""SQVI""","""Sequoia Vaccines, Inc.""","""$8.00 - $10.00""",2775000,9.0,"""Inc""",24.975
"""SNI""","""Shenni Holdings Limited""","""$4.00 - $6.00""",3000000,5.0,"""Limited""",15.0
…,…,…,…,…,…,…
"""DPAC""","""Deep Space Acquisition Corp. I""","""$10.00""",21000000,10.0,"""Acq.Corp""",210.0
"""GIF""","""GigCapital6, Inc.""","""$10.00""",20000000,10.0,"""Inc""",200.0
"""HYIV""","""Haymaker Acquisition Corp. IV""","""$10.00""",26100000,10.0,"""Acq.Corp""",261.0


In [5]:
pl.Config.set_float_precision(2)

(
    df.group_by('Company Class')
    .agg(c('Withdrawn Value (million)').sum())
).sort('Withdrawn Value (million)', descending=True)

Company Class,Withdrawn Value (million)
cat,f64
"""Acq.Corp""",4021.00
"""Inc""",2257.16
"""Other""",767.92
"""Limited""",549.73
"""Holdings""",75.00
"""Group""",33.79


# Question 2: [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)

What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?

The goal is to replicate the large-scale yfinance OHLCV data download and perform basic financial calculations on IPO stocks.

Steps:

1. Using the same approach as in Question 1, download the IPOs in 2024 from: https://stockanalysis.com/ipos/2024/
   Filter to keep only those IPOs before 1 June 2024 (first 5 months of 2024). ➤ You should have 75 tickers.
2. Use Code Snippet 7 to download daily stock data for those tickers (via yfinance).
   Make sure you understand how `growth_1d` ... `growth_365d`, and volatility columns are defined.
   Define a new column `growth_252d` representing growth after 252 trading days (~1 year),
   in addition to any other growth periods you already track.
3. Calculate the Sharpe ratio assuming a risk-free rate of 4.5%:

    stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']

4. Filter the DataFrame to keep data only for the trading day: '2025-06-06'.
   Compute descriptive statistics (e.g., .describe()) for these columns:
   - `growth_252`
   - Sharpe
   You should observe:
   - `growth_252d` is defined for 71 out of 75 stocks (some IPOs are too recent or data starts later).
   - Median `growth_252d` is approximately 0.75 (indicating a 25% decline), while mean is about 1.15,
     showing a bias towards high-growth companies pushing the average up.

5. Answer:
   - What is the median Sharpe ratio for these 71 stocks?
   - Note: Positive Sharpe means growth exceeding the risk-free rate of 4.5%.
   - [Additional] Do you observe the same top 10 companies when sorting by `growth_252d` versus sorting by Sharpe?

In [6]:
df = read_table('https://stockanalysis.com/ipos/2024/')
df

IPO Date,Symbol,Company Name,IPO Price,Current,Return
str,str,str,str,str,str
"""Dec 31, 2024""","""ONEG""","""OneConstruction Group Limited""","""$4.00""","""$3.73""","""-6.64%"""
"""Dec 27, 2024""","""PHH""","""Park Ha Biological Technology …","""$4.00""","""$19.80""","""395.00%"""
"""Dec 23, 2024""","""HIT""","""Health In Tech, Inc.""","""$4.00""","""$0.60""","""-85.08%"""
"""Dec 23, 2024""","""TDAC""","""Translational Development Acqu…","""$10.00""","""$10.28""","""2.80%"""
"""Dec 20, 2024""","""RANG""","""Range Capital Acquisition Corp…","""$10.00""","""$10.31""","""3.10%"""
…,…,…,…,…,…
"""Jan 18, 2024""","""CCTG""","""CCSC Technology International …","""$6.00""","""$1.08""","""-82.00%"""
"""Jan 18, 2024""","""PSBD""","""Palmer Square Capital BDC Inc.""","""$16.45""","""$14.20""","""-13.68%"""
"""Jan 12, 2024""","""SYNX""","""Silynxcom Ltd.""","""$4.00""","""$1.89""","""-52.75%"""


In [7]:
from datetime import date

df = (
    df.with_columns(
        c('IPO Date').str.strptime(pl.Date, '%B %d, %Y'),
        c('IPO Price').str.replace(r'\$', '').cast(pl.Float32, strict=False),
        c('Current').str.replace(r'\$', '').cast(pl.Float32, strict=False),
        c('Return').str.replace('%', '').cast(pl.Float32, strict=False),
    )
    .filter((c('IPO Date') < date(2024, 6, 1)) & c('Return').is_not_null())
)

In [8]:
import yfinance as yf
from pathlib import Path

def load_ohlc_history(symbol: str) -> pl.DataFrame:
    fname = Path.cwd() / f'{symbol.lower()}.csv'
    
    if fname.exists():
        df = (
            pl.read_csv(fname)
            .with_columns(
                Date=c('Date').str.strptime(pl.Datetime(time_unit='us'), '%Y-%m-%dT%H:%M:%S%.9f%z', strict=False),
                Symbol=pl.lit(symbol),
            )
        )
    else:
        df = (
            pl.DataFrame(yf.Ticker(symbol).history(period='max', interval='1d').reset_index())
            .with_columns(
                Date=c('Date').dt.date(),
                Symbol=pl.lit(symbol),
            )
        )
        df.write_csv(fname)   # cache the data to prevent API throttling problems during experimentations
    
    return df.sort('Date', descending=False)

In [9]:
from tqdm.auto import tqdm
import numpy as np

ohlc_df = (
    pl.concat([load_ohlc_history(symbol) for symbol in tqdm(df['Symbol'].to_list())])
    .lazy()
    .sort('Symbol', 'Date')
    .with_columns(
        Date=c('Date').dt.date(),
        growth_252d=c('Close').over('Symbol') / c('Close').shift(252).over('Symbol'),
        volatility=c('Close').rolling_std(30) * np.sqrt(252),
    )
    .with_columns(
        Sharpe=c('growth_252d') / c('volatility')
    )
).collect()
ohlc_df

  0%|          | 0/75 [00:00<?, ?it/s]

Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol,growth_252d,volatility,Sharpe
date,f64,f64,f64,f64,i64,f64,f64,str,f64,f64,f64
2024-02-07,12.09,12.47,11.88,12.43,12732800,0.00,0.00,"""AHR""",null,null,null
2024-02-08,12.25,12.45,12.23,12.28,1630300,0.00,0.00,"""AHR""",null,null,null
2024-02-09,12.24,12.65,12.18,12.34,2062100,0.00,0.00,"""AHR""",null,null,null
2024-02-12,13.12,13.69,12.32,12.37,1161200,0.00,0.00,"""AHR""",null,null,null
2024-02-13,12.37,12.75,12.24,12.56,1631700,0.00,0.00,"""AHR""",null,null,null
…,…,…,…,…,…,…,…,…,…,…,…
2025-06-09,4.35,4.50,3.78,3.89,172500,0.00,0.00,"""ZONE""",1.32,13.88,0.10
2025-06-10,3.90,3.95,3.61,3.79,79600,0.00,0.00,"""ZONE""",1.23,14.60,0.08
2025-06-11,3.73,4.10,3.50,3.72,132300,0.00,0.00,"""ZONE""",1.24,15.13,0.08


In [10]:
(
    ohlc_df.filter(c('Date') == date(2025, 6, 6))
    .select(
        growth_252d_median=c('growth_252d').median(),
        non_null_growth_252d=c('growth_252d').is_not_null().count(),
        non_nan_growth_252d=c('growth_252d').is_not_nan().count(),
        sharpe_median=c('Sharpe').median(),
    )
).transpose(include_header=True, header_name='Measure')

Measure,column_0
str,f64
"""growth_252d_median""",0.76
"""non_null_growth_252d""",75.00
"""non_nan_growth_252d""",71.00
"""sharpe_median""",0.10
